In [29]:
train_projects = [ # total: 13 point clouds (including bbox splitting)
    # {"proj_name": "2023-08-28_FW_EingangBauing.FwfProj", "bboxes" : []},                    # building
    {"proj_name": "2024-03-22_FW_Koenigshuegel.FwfProj", "bboxes" : []},                      # building
    # {"proj_name": "2024-04-05_FW_Westbahnhof_01.FwfProj", "bboxes" : []},                   # tunnel-bridge
    {"proj_name": "2024-04-05_FW_Westbahnhof_02.FwfProj", "bboxes" : []},                     # tunnel-bridge
    {"proj_name": "2024-04-05_FW_Westbahnhof_03.FwfProj", "bboxes" : []},                     # tunnel-bridge
    {"proj_name": "2024-04-05_FW_Westbahnhof_04.FwfProj", "bboxes" : []},                     # tunnel-bridge
    {"proj_name": "2024-04-05_FW_Westbahnhof_05.FwfProj", "bboxes" : []},                     # building
    {"proj_name": "2024-05-10_FW_RWTH_Zentrum_01.FwfProj", "bboxes" : []},                    # building
    {"proj_name": "2024-07-31_FW_Bruecke_Koenigstr.FwfProj", "bboxes" : [0,2]},               # bridge (steel+stone)
    # {"proj_name": "2024-07-31_FW_Bruecke_Turmstr.FwfProj", "bboxes" : []},                  # bridge (concrete/steel)
    {"proj_name": "2024-08-02_FW_Bruecke_A44_VerlautenheidenerStr.FwfProj", "bboxes" : []},   # bridge (concrete)
    {"proj_name": "2024-08-02_FW_Bruecke_Deltourserb.FwfProj", "bboxes" : []},                # bridge (concrete)
    {"proj_name": "2024-08-02_FW_Bruecke_Kasinostrasse.FwfProj", "bboxes" : [1]},             # bridge (concrete, steel, brick)
    {"proj_name": "2024-08-02_FW_Bruecke_RotheErde.FwfProj", "bboxes" : []},                  # bridge (steel)
    {"proj_name": "2024-08-02_FW_Bruecke_Rottstrasse.FwfProj", "bboxes" : []},                # bridge (concrete)
]
val_projects = [ # total: 5 point clouds (including bbox splitting)
    {"proj_name": "2023-08-28_FW_EingangBauing.FwfProj", "bboxes" : []},                    # building
    {"proj_name": "2024-04-05_FW_Westbahnhof_01.FwfProj", "bboxes" : []},                   # tunnel-bridge
    {"proj_name": "2024-07-31_FW_Bruecke_Koenigstr.FwfProj", "bboxes" : [1]},               # bridge (steel+stone)
    {"proj_name": "2024-08-02_FW_Bruecke_Kasinostrasse.FwfProj", "bboxes" : [0]},           # bridge (concrete, steel, brick)
    {"proj_name": "2024-07-31_FW_Bruecke_Turmstr.FwfProj", "bboxes" : []},                  # bridge (concrete/steel)


]

In [30]:
import numpy as np
class BBox:
    """Bounding box using the CloudCompare synthax
    """
    def __init__(self, orientation:np.ndarray, width: np.ndarray) -> None:
        # an easy way to retrieve these values is by using the cross-section tool in cloudcompare
        # so cross-section -> advanced -> orientation -> to clipboard
        self.orientation = orientation
        # from the same menu copy and paste the values under 'width' 
        self.width = width
        
    def cutout(self, xyz:np.ndarray) -> np.ndarray:
            # Inverse of the orientation matrix (rotation and translation)
            orientation_inv = np.linalg.inv(self.orientation)
            
            # Transform points into the bbox's local coordinate system
            transformed_xyz = (orientation_inv @ np.vstack((xyz.T, np.ones(xyz.shape[0])))).T
            
            # Get the half-dimensions of the bbox (since width is the full length)
            half_width = self.width / 2
            
            # Check if points lie within the bbox in local coordinates
            inside_bbox = (
                (np.abs(transformed_xyz[:, 0]) <= half_width[0]) &
                (np.abs(transformed_xyz[:, 1]) <= half_width[1]) &
                (np.abs(transformed_xyz[:, 2]) <= half_width[2])
            )
            
            # Return the indices of the points inside the bbox
            return np.where(inside_bbox)[0]


In [31]:
from utils.pointcloud import grid_subsample
import numpy as np
import torch

def grid_subsample_simple(xyz:np.ndarray, voxel_size:float, device='cuda'):
    """
    Wrapper using numpy arrays
    """
    sub = grid_subsample(torch.Tensor(xyz).to(device=device), voxel_size) # n -> m number of points
    sub['points'] = sub['points'].to(dtype=torch.float64).detach().cpu().numpy()
    sub['inv_inds'] = sub['inv_inds'].detach().cpu().numpy()
    return sub

In [32]:
x = np.random.random((1000,3))
sub = grid_subsample_simple(x, 0.04)
sub['points']

array([[0.01347592, 0.0016289 , 0.25991973],
       [0.02035444, 0.03605291, 0.51316279],
       [0.01292201, 0.05100558, 0.98834091],
       ...,
       [0.99641997, 0.88954228, 0.12952007],
       [0.99063897, 0.94944626, 0.3874771 ],
       [0.98243093, 0.9228611 , 0.93124831]])

In [33]:
from plyfile import PlyData, PlyElement
import pandas as pd
import numpy as np
from glob import glob
import json
import os
from omegaconf import OmegaConf
from scipy.spatial import KDTree
from torch.utils.data import Dataset, DataLoader
from jakteristics import compute_features

import torch

from utils.pointcloud import grid_subsample

class FwfDataset(Dataset):
    def __init__(self,
                 proj_search_pattern,
                 proj_query_list,
                 return_fields_input = None,
                 label_names = None,
                 return_waveform = True,
                 query_grid_size = None,
                 subsample_on_gpu = True
                 ):
        
        super(Dataset,self).__init__()
        
        self.proj_query_list = proj_query_list
        self.return_fields_input = return_fields_input
        self.return_waveform = return_waveform

        self.projects = list()
        
        # keep track of point cloud sizes
        self.proj_lens = []
        
        # go through search pattern
        for fwf_prof_fp in glob(proj_search_pattern):
            proj_name = os.path.basename(fwf_prof_fp)
            
            # only get projects if they are on the project list
            if proj_name not in [p['proj_name'] for p in self.proj_query_list]:
                continue
            else:
                bbox_queries = [p['bboxes'] for p in self.proj_query_list if p['proj_name']==proj_name][0]

                
            print(f"Loading '{proj_name}'; Bounding box IDs = {bbox_queries if len(bbox_queries) else 'default'}")

            # load the data
            pcd = pd.DataFrame(PlyData.read(list(glob(os.path.join(fwf_prof_fp,'labeled','*pointcloud.ply')))[0]).elements[0].data)
            wfm = np.load(list(glob(os.path.join(fwf_prof_fp,'labeled','*waveform.npy')))[0])
            meta = json.load(open(list(glob(os.path.join(fwf_prof_fp,'labeled','*metadata.json')))[0],"r"))


            # get scan positions
            if 'scanId=000' in meta['scan_positions'].keys():
                sop = np.array([meta['scan_positions'][f'scanId={si:03}']['sop'] for si in range(len(meta['scan_positions']))])
            else:
                # handle case where only one scan position is in the metadata and it's f.s.r. labeled 'scanId=001' instead of 'scanID=000'
                sop = np.array([meta['scan_positions']['scanId=001']['sop']])



            # field names (constants)
            riegl_feat_names = [ 'riegl_reflectance','riegl_amplitude', 'riegl_deviation', 'riegl_targetIndex','riegl_targetCount']
            # label_names = ['labels_0', 'labels_1', 'labels_2', 'labels_3']
            self.label_names = label_names


            # get full xyz
            xyz_defaultBbox = pcd[['x','y','z']].to_numpy()
            

                
            # save project as dict
            if len(bbox_queries)==0:
                # handle default case
                sub = grid_subsample_simple(xyz_defaultBbox,query_grid_size, 'cuda' if subsample_on_gpu else 'cpu')
                kd_tree = KDTree(xyz_defaultBbox)
                _, sub_ids = kd_tree.query(sub['points'])
                self.projects.append(dict(
                    proj_name=f"{proj_name}::defaultBbox",
                    xyz = xyz_defaultBbox,
                    wfm = wfm,
                    sop = sop,
                    rgb = pcd[['Red','Green','Blue']].to_numpy(),
                    riegl_feats = pcd[riegl_feat_names].to_numpy(),
                    labels = pcd[self.label_names].to_numpy()[sub_ids], # labels need to be subsampeld 
                    sop_ids = pcd['scan_id'].to_numpy(),
                    kd_tree = kd_tree,
                    xyz_sub = sub['points'],
                    sub_inv = sub['inv_inds']
                ))
                self.proj_lens.append(sub['points'].shape[0])
            else:
                # handle region bboxes case
                for bbox_i in bbox_queries:
                    bbox_meta = meta['bboxes'][f'bboxId={bbox_i:03}']
                    bbox = BBox(orientation=np.array(bbox_meta['orientation']), width=np.array(bbox_meta['width']))
                    subcloud_mask=bbox.cutout(xyz_defaultBbox)
                    xyz_masked = xyz_defaultBbox[subcloud_mask]
                    sub = grid_subsample_simple(xyz_masked,query_grid_size, 'cuda' if subsample_on_gpu else 'cpu')
                    kd_tree = KDTree(xyz_masked)
                    _, sub_ids = kd_tree.query(sub['points'])
                    self.projects.append(dict(
                        proj_name=f"{proj_name}::bboxId={bbox_i:03}",
                        xyz = xyz_masked,
                        wfm = wfm[subcloud_mask],
                        sop = sop,
                        rgb = pcd[['Red','Green','Blue']].to_numpy()[subcloud_mask],
                        riegl_feats = pcd[riegl_feat_names].to_numpy()[subcloud_mask],
                        labels = pcd[self.label_names].to_numpy()[subcloud_mask][sub_ids], # labels need to be subsampeld
                        sop_ids = pcd['scan_id'].to_numpy()[subcloud_mask],
                        kd_tree = KDTree(xyz_masked),
                        xyz_sub = sub['points'],
                        sub_inv = sub['inv_inds']
                    ))
                    self.proj_lens.append(sub['points'].shape[0])
                    
            # calculate the cumulative sum of the point cloud sizes

            # break
        self.proj_lens_cumsum = np.cumsum(self.proj_lens)
    
    def compute_neibors_knn(self, k:int):
        for proj in self.projects:
            print(f"Computing neibors for '{proj['proj_name']}' @ k={k}")
            dists, neib_ids = proj['kd_tree'].query(proj['xyz_sub'], k)
            proj['neibors'] = neib_ids
    

   
    def compute_normals_knn(self):
        for proj in self.projects:
            k = proj['neibors'].shape[1]
            print(f"Computing normals for '{proj['proj_name']}' @ k={k}")
            neibs_xyz = proj['xyz'][proj['neibors']]

            means = neibs_xyz.mean(axis=1, keepdims=True)
            neibs_xyz -= means
            cov = (neibs_xyz.transpose([0,2,1]) @ neibs_xyz) / (k-1)
            eigenvals, eigenvecs = np.linalg.eigh(cov)
            # get non-flipped normals
            normals = eigenvecs[:, :, 0]

            # upsample normals to full resolution
            normals = normals[proj['sub_inv']]
            
            # move all points to scanner CS
            points_origin_scanPos = proj['sop'][proj['sop_ids']][:,:3,3]
            xyz_scannerCs = proj['xyz'] - points_origin_scanPos
            signs = np.sign(np.squeeze(xyz_scannerCs[:,None,:] @ normals [:,:,None])) * -1
            normals *= signs[:,None]
            
            proj['normals'] = normals




            
            
    # def compute_normals_spherical_subsample(self, r = 0.08, voxel_size = 0.03, device='cuda'):
        
    #     for proj in self.projects:
    #         print(f"Computing normals for '{proj['proj_name']}' @ r={r:.3f}m and voxel_size = {voxel_size:.3f}")
    #         print("\tDEBUG: subsampling")
    #         sub = grid_subsample(torch.Tensor(proj['xyz']).to(device=device), voxel_size) # n -> m number of points
    #         sub['points'] = sub['points'].to(dtype=torch.float64).detach().cpu().numpy()
    #         sub['inv_inds'] = sub['inv_inds'].detach().cpu().numpy()
    #         _,sub_ids = proj['kd_tree'].query(sub['points'])
    #         print("\tDEBUG: computing feats")
    #         sub_normals = compute_features(sub['points'],search_radius=0.05,feature_names=['nx','ny','nz'])

    #         # flip the normals
    #         print("\tDEBUG: flipping the subsampled normals")
    #         sub_xyz_scannerCs = proj['xyz'][sub_ids] - proj['sop'][proj['sop_ids']][:,3,:3][sub_ids]
    #         sub_signs = np.sign(np.squeeze(sub_xyz_scannerCs[:,None,:] @ sub_normals[:,:,None]))*-1
    #         sub_normals *= sub_signs[:,None]

    #         print("\tDEBUG: reprojecting")
    #         proj['normals'] = sub_normals[sub['inv_inds']]



    
    
    # def compute_normals_spherical(self, r = 0.08):
    #     for proj in self.projects:
    #         print(f"Computing normals for '{proj['proj_name']}' @ r={r:.3f}m")
            
    #         # get non-oriented normals
    #         normals = compute_features(proj['xyz'],search_radius=0.05,feature_names=['nx','ny','nz'])
    #         # orient the normals to corresponding scanner position
    #         xyz_scannerCs = proj['xyz'] - proj['sop'][proj['sop_ids']][:,3,:3]
    #         signs = np.sign(np.squeeze(xyz_scannerCs[:,None,:] @ normals [:,:,None])) * -1
    #         normals *= signs[:,None]
            
            
            
    def compute_incAngles(self):
        for proj in self.projects:
            print(f"Computing incidence angles for '{proj['proj_name']}'")
            xyz_scannerCs = proj['xyz'] - proj['sop'][proj['sop_ids']][:,:3,3]
            proj['incAngles']= np.arccos(np.squeeze((proj['normals'][:,None,:] @ xyz_scannerCs[...,None])) / \
                (np.linalg.norm(xyz_scannerCs,axis=-1) * np.linalg.norm(proj['normals'],axis=-1)))
            proj['distanceFromScanner'] = np.linalg.norm(xyz_scannerCs, axis=-1)
            
    
    def __getitem__(self, index):
        # get proj index first
        proj_idx = np.argwhere(index<self.proj_lens_cumsum)[0][0]
        size_prev = self.proj_lens_cumsum[proj_idx-1] if proj_idx > 0 else 0
        residual_idx = index - size_prev
        
        # get neibors of point at index
        neibs = self.projects[proj_idx]['neibors'][residual_idx]
        
        if self.return_fields_input == None:
            # extract all fields
            return_dict = dict(
                inputs = np.concatenate([
                self.projects[proj_idx]['xyz'][neibs],
                self.projects[proj_idx]['rgb'][neibs],
                self.projects[proj_idx]['riegl_feats'][neibs],
                self.projects[proj_idx]['normals'][neibs],
                self.projects[proj_idx]['incAngles'][neibs][:,None],
                self.projects[proj_idx]['distanceFromScanner'][neibs][:,None],
                ], axis=-1),
            )
        else:
            # only extract specific fields
            return_dict = dict(inputs = np.concatenate([self.projects[proj_idx][f][neibs] for f in self.return_fields_input]))
        
        # add waveforms and labels
        return_dict.update(dict(
            wfm = self.projects[proj_idx]['wfm'][neibs] if self.return_waveform else None,
            labels = self.projects[proj_idx]['labels'][residual_idx],
        ))
        return return_dict
        
    
    def __len__(self):
        return np.sum([p['xyz_sub'].shape[0] for p in self.projects])
        




In [34]:
aa = dict(
    a = 1,
    b = 2
)
bb = dict(
    c = 3,
    d = 4
)

aa | bb

{'a': 1, 'b': 2, 'c': 3, 'd': 4}

In [35]:
for i in train_ds[0].items():
    print(f"{i[0]:.<30}", i[1].shape)

xyz........................... (20, 3)
wfm........................... (20, 32)
rgb........................... (20, 3)
riegl_feats................... (20, 5)
labels........................ (4,)
normals....................... (20, 3)
incAngles..................... (20,)
distanceFromScanner........... (20,)


In [36]:
from torch import nn

class FGFeatNetwork (nn.Module):
    def __init__(self,
                 num_input_feats,
                 label_structure = {'labels_0':3, 'labels_1':10, 'labels_2':12, 'labels_3':18}
                 ):
        super(FGFeatNetwork, self).__init__()
        
        self.label_structure = label_structure
        
        # Pointwise feats
        self.mlp1 = nn.Sequential(
            nn.Linear(num_input_feats, 64), nn.ReLU(),
            nn.Linear(64, 128), nn.ReLU(),
            nn.Linear(128, 256), nn.ReLU()
        )
        self.wf_conv = nn.Sequential(
            # FIXME: check conv and maxpool kernel sizes and strides
            nn.Conv1d(in_channels=1, out_channels=16, kernel_size=3), nn.RReLU(), nn.MaxPool1d(2), # activations 32 -> 16
            nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3), nn.RReLU(), nn.MaxPool1d(2),# activations 16 -> 8
            nn.Conv1d(in_channels=32, out_channels=64, kernel_size=3), nn.RReLU(), nn.MaxPool1d(2),# activations 8 -> 4
            # after concat. activation shape = 4 * 64 = 256
        )
        # TODO: Try out inverse bottleneck?
        # TODO: Network size might be an overkill / unfeasable for the task
        # MLP after concat with WFM feats
        self.mlp2 = nn.Sequential(
            nn.Linear(512, 512), nn.ReLU(),
            nn.Linear(512, 1024), nn.ReLU(),
        )
        
        # decoder
        self.mlp3 = nn.Sequential(
            nn.Linear(1024, 512), nn.ReLU(),
            nn.Linear(512, 256), nn.ReLU(),
            nn.Linear(256, 128), nn.ReLU(),
        )
        
        # classifier
        self.classifier = nn.ModuleDict({k:nn.Linear(128,v) for k,v in self.label_structure.items()})
        
    def forward(self, x):
        pw_feats = self.mlp1(x['inputs'])
        wf_feats = self.wf_conv(x['wfm'])
        joined_feats = torch.cat([pw_feats, wf_feats],dim = -1)
        joined_feats = self.mlp3(joined_feats)
        

        raise NotImplementedError
        
        

In [37]:
input_fields = [
    # ('labels',4),
    # ('wfm',32),
    ('xyz',3),
    ('rgb',3),
    ('riegl_feats',5),
    ('normals',3),
    ('incAngles',1),
    ('distanceFromScanner',1)
]

16

In [45]:
model = FGFeatNetwork(
    num_input_feats = train_ds[0]['inputs'].shape[-1],
    label_structure = {
        "labels_0":3,
        "labels_1":8,
        # "labels_2":18,
        # "labels_3":18
})


In [43]:
for i in batch.items():
    print(f"{i[0]:.<30}", i[1].shape)

inputs........................ torch.Size([32, 20, 16])
wfm........................... torch.Size([32, 20, 32])
labels........................ torch.Size([32, 2])


In [42]:
train_dl = DataLoader(train_ds, batch_size=32)

i = 0
for batch in train_dl:
    break
    print(i)
    i+=1
    if i > 30:
        break

In [46]:
# data_dir = r"D:\Projekte\GIA_220412_PCS\02_Datasets\FullWaveForm\full_waveform_software\FullWaveformParse_mass\output\FWF_Aachen_labeled\*.FwfProj"              
proj_search_pattern = "../../02_Datasets/FWF_Aachen_labeled/*.FwfProj"      
train_ds = FwfDataset(
    proj_search_pattern=proj_search_pattern, 
    proj_query_list=train_projects, 
    query_grid_size=0.4,
    return_fields_input = None,
    label_names = ['labels_0', 'labels_1'],
    return_waveform = True,
    subsample_on_gpu = True
)




Loading '2024-03-22_FW_Koenigshuegel.FwfProj'; Bounding box IDs = default
Loading '2024-04-05_FW_Westbahnhof_02.FwfProj'; Bounding box IDs = default
Loading '2024-04-05_FW_Westbahnhof_03.FwfProj'; Bounding box IDs = default
Loading '2024-04-05_FW_Westbahnhof_04.FwfProj'; Bounding box IDs = default
Loading '2024-04-05_FW_Westbahnhof_05.FwfProj'; Bounding box IDs = default
Loading '2024-05-10_FW_RWTH_Zentrum_01.FwfProj'; Bounding box IDs = default
Loading '2024-07-31_FW_Bruecke_Koenigstr.FwfProj'; Bounding box IDs = [0, 2]
Loading '2024-08-02_FW_Bruecke_A44_VerlautenheidenerStr.FwfProj'; Bounding box IDs = default
Loading '2024-08-02_FW_Bruecke_Deltourserb.FwfProj'; Bounding box IDs = default
Loading '2024-08-02_FW_Bruecke_Kasinostrasse.FwfProj'; Bounding box IDs = [1]
Loading '2024-08-02_FW_Bruecke_RotheErde.FwfProj'; Bounding box IDs = default
Loading '2024-08-02_FW_Bruecke_Rottstrasse.FwfProj'; Bounding box IDs = default


In [47]:
train_ds.compute_neibors_knn(k=20)
train_ds.compute_normals_knn()
train_ds.compute_incAngles()

Computing neibors for '2024-03-22_FW_Koenigshuegel.FwfProj::defaultBbox' @ k=20
Computing neibors for '2024-04-05_FW_Westbahnhof_02.FwfProj::defaultBbox' @ k=20
Computing neibors for '2024-04-05_FW_Westbahnhof_03.FwfProj::defaultBbox' @ k=20
Computing neibors for '2024-04-05_FW_Westbahnhof_04.FwfProj::defaultBbox' @ k=20
Computing neibors for '2024-04-05_FW_Westbahnhof_05.FwfProj::defaultBbox' @ k=20
Computing neibors for '2024-05-10_FW_RWTH_Zentrum_01.FwfProj::defaultBbox' @ k=20
Computing neibors for '2024-07-31_FW_Bruecke_Koenigstr.FwfProj::bboxId=000' @ k=20
Computing neibors for '2024-07-31_FW_Bruecke_Koenigstr.FwfProj::bboxId=002' @ k=20
Computing neibors for '2024-08-02_FW_Bruecke_A44_VerlautenheidenerStr.FwfProj::defaultBbox' @ k=20
Computing neibors for '2024-08-02_FW_Bruecke_Deltourserb.FwfProj::defaultBbox' @ k=20
Computing neibors for '2024-08-02_FW_Bruecke_Kasinostrasse.FwfProj::bboxId=001' @ k=20
Computing neibors for '2024-08-02_FW_Bruecke_RotheErde.FwfProj::defaultBbox'

(20, 16)

In [31]:
batch['xyz'].shape

torch.Size([32, 20, 3])

In [26]:
train_dl = DataLoader(train_ds, batch_size=32)

i = 0
for batch in train_dl:
    print(i)
    i+=1
    if i > 30:
        break

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30


In [9]:
# visualize normals
# proj = train_ds.projects[7]
for i, proj in enumerate(train_ds.projects):
    PlyData([PlyElement.describe(pd.DataFrame(np.concatenate([proj['xyz'],proj['normals'],np.squeeze(proj['incAngles'])[:,None],proj['distanceFromScanner'][:,None], proj['sop_ids'][:,None]], axis=1),columns=['x','y','z','nx','ny','nz','incAngle','distanceFromScanner','sop_ids']).to_records(index=False),'vertex')]).write(f"./_temp/proj_{i:03}.ply")